# Caso simplesmente Apoiado

Conforme Timoshenko, a equação da deflexão da placa é computada com a seguinte equação,

\begin{equation}
 w_{max}=\frac{5}{384}\frac{qa^4}{D}-\frac{4qa^4}{\pi^5D}\sum_{m=1,3,5,\cdots}^{\infty}\frac{(-1)^{(m-1)/2}}{m^5}\left(\frac{\alpha_m \tanh \alpha_m + 2}{2 \cosh \alpha_m} \right)
\end{equation}

Nota-se que a equação é denotada como uma soma infinita convergente, então pode-se computar a soma previamente utilizando uma linguagem de programação, no caso o Python. Assim o código abaixo calculará o valor dessa soma.

In [372]:
import sympy as sp
import numpy as np

# Símbolos
a, q, D, b = sp.symbols('a q D b')
m = sp.symbols('m', integer=True)

# Define alpha_m
alpha = (m * sp.pi * b) / (2 * a)

# Termo da série
sign = (-1)**((m - 1)//2)
term1 = 1 / m**5
term2 = (alpha * sp.tanh(alpha) + 2) / (2*sp.cosh(alpha))

summand = sign * term1 * term2

# Valores de lambda
lambda_vals = np.linspace(1, 2.0, 21)
w_normalized = []

# Loop para cada lambda
for λ in lambda_vals:
    summand_l = summand.subs(b, λ * a)
    partial_sum = sum(summand_l.subs(m, mi) for mi in [1, 3, 5, 7, 9])
    num_val = partial_sum.evalf()

    # Cálculo da deflexão
    w = (5/384)*(q*a**4/D)-(4*q*a**4/(sp.pi**5*D))*num_val
    w_norm = w.evalf() * D / (a**4 * q)
    w_normalized.append(w_norm) #finalmente, armazena o valor normalizado



In [373]:
w_normalized

[0.00406235266062708,
 0.00446759395629202,
 0.00486895694652180,
 0.00526395230313697,
 0.00565053001075757,
 0.00602704367869330,
 0.00639220942680839,
 0.00674506285794067,
 0.00708491657890747,
 0.00741131989068944,
 0.00772402162333273,
 0.00802293661625737,
 0.00830811600929865,
 0.00857972128394266,
 0.00883800185121825,
 0.00908327590034574,
 0.00931591418276839,
 0.00953632639592518,
 0.00974494983983497,
 0.00994224003981589,
 0.0101286630552058]

In [374]:
h, nu, E = sp.symbols('h nu E')

D=(E*h**3)/(12*(1-nu**2))

D = D.subs(E, 25000) # 25000MPa
D = D.subs(h, 0.08) # 8cm de altura da laje
D = D.subs(nu, 0.2) # Coeficiente de poisson do concreto

Factor_evalued=1*1**4/D.evalf()
Factor_evalued

w_real=[]

for i in range(0, len(w_normalized)):
    w_real.append(w_normalized[i]*Factor_evalued*1000)
    print(f"({lambda_vals[i]:,.2f},{w_real[i]:,.5f})")

(1.00,3.65612)
(1.05,4.02083)
(1.10,4.38206)
(1.15,4.73756)
(1.20,5.08548)
(1.25,5.42434)
(1.30,5.75299)
(1.35,6.07056)
(1.40,6.37642)
(1.45,6.67019)
(1.50,6.95162)
(1.55,7.22064)
(1.60,7.47730)
(1.65,7.72175)
(1.70,7.95420)
(1.75,8.17495)
(1.80,8.38432)
(1.85,8.58269)
(1.90,8.77045)
(1.95,8.94802)
(2.00,9.11580)


In [375]:
# Guarda a expressão simbólica original
D2_original = (E*h**3)/(12*(1-nu**2))

espessuras = [0.05, 0.06, 0.08, 0.1, 0.12, 0.14, 0.18]

# Substituições que não mudam
D2_parcial = D2_original.subs(E, 25000).subs(nu, 0.2)

for espessura in espessuras:
    D2 = D2_parcial.subs(h, espessura)   # agora funciona!
    Factor_evalued = 1 * 1**4 / D2.evalf()

    print(f"({espessura*100:.2f}, {w_normalized[0]*Factor_evalued*1000:.2f})") #(espessura em cm , deflexão em mm)


(5.00, 14.98)
(6.00, 8.67)
(8.00, 3.66)
(10.00, 1.87)
(12.00, 1.08)
(14.00, 0.68)
(18.00, 0.32)


In [376]:
%reset_selective -f ^((?!w_normalized).)*$

# Caso biengastado e biapoiado

Conforme Timoshenko, a equação da deflexão da placa é computada com a seguinte equação,

\begin{equation}
 w_{1}=\frac{4qa^4}{\pi^5D}\sum_{m=1,3,5,\cdots}^{\infty}\frac{(-1)^{(m-1)/2}}{m^5}\left( 1 - \frac{\alpha_m \tanh \alpha_m + 2}{2 \cosh \alpha_m} \right)
\end{equation}

Devido aos engastes, há uma reação de momento nas bordas, e o deslocamento total é dado como a superposição de soluções. Ou seja, $w_1$ é montada adicionando momentos nas bordas da placa, e o deslocamento real é dado por $w=w+w_1$, onde $w$ é o deslocamento do caso simplesmente apoiado.

In [377]:
import sympy as sp
import numpy as np

# Símbolos
a, q, D, b = sp.symbols('a q D b')
m = sp.symbols('m', integer=True)

# Define alpha_m
alpha = (m * sp.pi * b) / (2 * a)

# Termo da série
sign = (-1)**((m - 1)//2)
term1 = 1 / m**5
term2 = alpha * sp.tanh(alpha) / sp.cosh(alpha)
numerator = alpha - sp.tanh(alpha) * (1 + alpha * sp.tanh(alpha))
denominator = alpha - sp.tanh(alpha) * (alpha * sp.tanh(alpha) - 1)
term3 = numerator / denominator

summand = sign * term1 * term2 * term3

# Valores de lambda
lambda_vals = np.linspace(1, 2.0, 21)
w1_normalized = []

# Loop para cada lambda
for λ in lambda_vals:
    summand_l = summand.subs(b, λ * a)
    partial_sum = sum(summand_l.subs(m, mi) for mi in [1, 3, 5, 7, 9])
    num_val = partial_sum.evalf()

    # Cálculo da deflexão
    w1 = (2*q*a**4)/(sp.pi**5*D)*num_val
    w1_norm = w1.evalf() * D / (a**4 * q)
    w1_normalized.append(w1_norm) #finalmente, armazena o valor normalizado

In [378]:
h, nu, E = sp.symbols('h nu E')

D=(E*h**3)/(12*(1-nu**2))

D = D.subs(E, 25000) # 25000MPa
D = D.subs(h, 0.08) # 8cm de altura da laje
D = D.subs(nu, 0.2) # Coeficiente de poisson do concreto

Factor_evalued=1*1**4/D.evalf()
Factor_evalued

w_real=[]

for i in range(0, len(w_normalized)):
    w_real.append((w_normalized[i]+w1_normalized[i])*Factor_evalued*1000)
    print(f"({lambda_vals[i]:,.2f},{w_real[i]:,.5f})")

(1.00,1.72542)
(1.05,1.99281)
(1.10,2.27502)
(1.15,2.56984)
(1.20,2.87492)
(1.25,3.18792)
(1.30,3.50650)
(1.35,3.82842)
(1.40,4.15155)
(1.45,4.47394)
(1.50,4.79380)
(1.55,5.10957)
(1.60,5.41984)
(1.65,5.72346)
(1.70,6.01943)
(1.75,6.30694)
(1.80,6.58536)
(1.85,6.85420)
(1.90,7.11314)
(1.95,7.36194)
(2.00,7.60050)


# Caso 1 engaste e 3 apoios

A equação da deflexão da placa é computada com a seguinte equação,

\begin{equation}
 (w1)_{x=a/2,y=0}=\frac{a^2}{4\pi^2D}\sum_{m=1,3,5,\cdots}^{\infty}{\frac{(-1)^{(m-1)/2}}{m^2}\frac{E_m\alpha_m \tanh \alpha_m}{\cosh \alpha_m}}
\end{equation}

Onde

\begin{equation}
 E_m=-\frac{8qa^2}{\pi^3m^3}\frac{\alpha_m-\tanh \alpha_m (1+\alpha_m \tanh \alpha_m)}{\alpha_m \tanh^2\alpha_m-\tanh alpha_m + \alpha_m \coth^2\alpha_m-\coth \alpha_m - 2\alpha_m}
\end{equation}

Da mesma forma que o problema anterior, o deslocamento $w$ é computado como superposição da equação acima com o caso simplesmenete apoiado, isto é $w=w+w_1$.

In [379]:
%reset_selective -f ^((?!(w1_normalized|w_normalized)).)*$

In [380]:
import sympy as sp
import numpy as np

# Símbolos
a, q, D, b = sp.symbols('a q D b')
m = sp.symbols('m', integer=True)

# Define alpha_m
alpha = (m * sp.pi * b) / (2 * a)

# Termo da série
sign = (-1)**((m - 1)//2)
term1 = 1 / m**2
term2 = (alpha * sp.tanh(alpha)) / sp.cosh(alpha)
Em1_term=-8*q*a**2/(sp.pi**3*m**3)
Em2_numerator=alpha-sp.tanh(alpha)*(1+alpha*sp.tanh(alpha))
Em2_denominator=alpha*(sp.tanh(alpha))**2-sp.tanh(alpha)+alpha*(sp.coth(alpha))**2-sp.coth(alpha)-2*alpha
Em=Em1_term*(Em2_numerator/Em2_denominator)

summand = sign * term1 * term2 * Em

# Valores de lambda
lambda_vals = np.linspace(1, 2.0, 21)
w_1c_3ss_normalized = []

# Loop para cada lambda
for λ in lambda_vals:
    summand_l = summand.subs(b, λ * a)
    partial_sum = sum(summand_l.subs(m, mi) for mi in [1, 3, 5, 7, 9])
    num_val = partial_sum.evalf()

    # Cálculo da deflexão
    w = (a**2)/(4*sp.pi**2*D)*num_val
    w_norm = w.evalf() * D / (a**4 * q)
    w_1c_3ss_normalized.append(w_norm) #finalmente, armazena o valor normalizado

In [381]:
w_normalized

[0.00406235266062708,
 0.00446759395629202,
 0.00486895694652180,
 0.00526395230313697,
 0.00565053001075757,
 0.00602704367869330,
 0.00639220942680839,
 0.00674506285794067,
 0.00708491657890747,
 0.00741131989068944,
 0.00772402162333273,
 0.00802293661625737,
 0.00830811600929865,
 0.00857972128394266,
 0.00883800185121825,
 0.00908327590034574,
 0.00931591418276839,
 0.00953632639592518,
 0.00974494983983497,
 0.00994224003981589,
 0.0101286630552058]

In [382]:
w_1c_3ss_normalized

[-0.00127685866366706,
 -0.00132214319903950,
 -0.00135503295983663,
 -0.00137616646956650,
 -0.00138641139706737,
 -0.00138678319019583,
 -0.00137837644454383,
 -0.00136230973241721,
 -0.00133968334514558,
 -0.00131154863034373,
 -0.00127888722249650,
 -0.00124259835882540,
 -0.00120349254635389,
 -0.00116229002429859,
 -0.00111962269280704,
 -0.00107603841705890,
 -0.00103200684158810,
 -0.000987926050702215,
 -0.000944129581715172,
 -0.000900893437711395,
 -0.000858442857810093]

In [383]:
h, nu, E = sp.symbols('h nu E')

D=(E*h**3)/(12*(1-nu**2))

D = D.subs(E, 25000) # 25000MPa
D = D.subs(h, 0.08) # 8cm de altura da laje
D = D.subs(nu, 0.2) # Coeficiente de poisson do concreto

Factor_evalued=1*1**4/D.evalf()
Factor_evalued

w_real=[]

for i in range(0, len(w_1c_3ss_normalized)):
    w_real.append((w_normalized[i]+w_1c_3ss_normalized[i])*Factor_evalued*1000)
    print(f"({lambda_vals[i]:,.2f},{w_real[i]:,.5f})")

(1.00,2.50694)
(1.05,2.83091)
(1.10,3.16253)
(1.15,3.49901)
(1.20,3.83771)
(1.25,4.17623)
(1.30,4.51245)
(1.35,4.84448)
(1.40,5.17071)
(1.45,5.48979)
(1.50,5.80062)
(1.55,6.10230)
(1.60,6.39416)
(1.65,6.67569)
(1.70,6.94654)
(1.75,7.20651)
(1.80,7.45552)
(1.85,7.69356)
(1.90,7.92074)
(1.95,8.13721)
(2.00,8.34320)


In [384]:
%reset_selective -f ^((?!(w1_normalized|w_normalized|w_1c_3ss_normalized)).)*$

# Caso totalmente engastado (4 bordas engastadas)

Por C.E. Ìmrak e İ. Gerdemeli, tem-se tabelados valores de deflexão com $\nu=0.3$, então utilizando os valores $(\lambda,w_{norm})$ como:

(1.0, 0.00126725)
(1.2, 0.00172833)
(1.4, 0.00207217)
(1.6, 0.00230399)
(1.8, 0.0024989)
(2.0, 0.00253625)

Pode-se calcular os correspondentes valores de $w=\frac{qa^4}{D}*w_{norm}$

In [385]:
import sympy as sp
import numpy as np


w=[0.00126725, 0.00172833, 0.00207217, 0.00230399, 0.0024989, 0.00253625]
lambda_vector=[1,1.2,1.4,1.6,1.8,2]

h, nu, E = sp.symbols('h nu E')

D=(E*h**3)/(12*(1-nu**2))

D = D.subs(E, 25000) # 25000MPa
D = D.subs(h, 0.08) # 8cm de altura da laje
D = D.subs(nu, 0.2) # Coeficiente de poisson do concreto

Factor_evalued=1*1**4/D.evalf()
Factor_evalued

w_real=[]

for i in range(0, len(w)):
    w_real.append((w[i]*(1*1/D.evalf())*1000))
    print(f"({lambda_vector[i]:,.2f},{w_real[i]:,.5f})")

(1.00,1.14052)
(1.20,1.55550)
(1.40,1.86495)
(1.60,2.07359)
(1.80,2.24901)
(2.00,2.28262)


NBR 6118


In [386]:
import math

f = 0
p = 1
lx = 1
E = 25000
h = 0.08

# α de cada caso (extraídos da tabela)
alpha_caso1 = [
    4.67, 5.17, 5.64, 6.09, 6.52, 6.95, 7.36, 7.76, 8.14, 8.51,
    8.87, 9.22, 9.54, 9.86, 10.15, 10.43, 10.71, 10.96, 11.21, 11.44,
    11.68, 15.35
]

alpha_caso2 = [
    3.20, 3.61, 4.04, 4.47, 4.91, 5.34, 5.77, 6.21, 6.62, 7.02,
    7.41, 7.81, 8.17, 8.52, 8.87, 9.19, 9.52, 9.82, 10.11, 10.39,
    10.68, 15.35
]

alpha_caso9 = [
    1.46, 1.60, 1.70, 1.80, 1.87, 1.98, 2.10, 2.20, 2.30, 2.37,
    2.45, 2.51, 2.57, 2.63, 2.68, 2.72, 2.76, 2.80, 2.85, 2.89,
    2.92, 3.07
]

alpha_caso5 = [
    2.42, 2.67, 2.91, 3.12, 3.34, 3.55, 3.73, 3.92, 4.08, 4.23,
    4.38, 4.53, 4.65, 4.77, 4.88, 4.97, 5.07, 5.16, 5.23, 5.31,
    5.39, 6.38
]

alpha_caso6 = [
    2.21, 2.55, 2.92, 3.29, 3.67, 4.07, 4.48, 2.69, 2.75, 2.80,
    2.84, 2.86, 2.89, 2.94, 2.97, 3.02, 3.07, 3.12, 3.17, 3.21,
    3.24, 3.07
]


def calcular(alpha, nome):
    print(f"\n=== {nome} ===")
    print("alpha\tf(mm)")
    for a in alpha:
        f = p * (lx ** 4) * a / (E * (h ** 3) * 100)
        print(f"({a}, {f * 1000:.4f})")


calcular(alpha_caso1, "CASO 1")
calcular(alpha_caso2, "CASO 2")
calcular(alpha_caso9, "CASO 9")
calcular(alpha_caso5, "CASO 5")
calcular(alpha_caso6, "CASO 6")



=== CASO 1 ===
alpha	f(mm)
(4.67, 3.6484)
(5.17, 4.0391)
(5.64, 4.4062)
(6.09, 4.7578)
(6.52, 5.0937)
(6.95, 5.4297)
(7.36, 5.7500)
(7.76, 6.0625)
(8.14, 6.3594)
(8.51, 6.6484)
(8.87, 6.9297)
(9.22, 7.2031)
(9.54, 7.4531)
(9.86, 7.7031)
(10.15, 7.9297)
(10.43, 8.1484)
(10.71, 8.3672)
(10.96, 8.5625)
(11.21, 8.7578)
(11.44, 8.9375)
(11.68, 9.1250)
(15.35, 11.9922)

=== CASO 2 ===
alpha	f(mm)
(3.2, 2.5000)
(3.61, 2.8203)
(4.04, 3.1562)
(4.47, 3.4922)
(4.91, 3.8359)
(5.34, 4.1719)
(5.77, 4.5078)
(6.21, 4.8516)
(6.62, 5.1719)
(7.02, 5.4844)
(7.41, 5.7891)
(7.81, 6.1016)
(8.17, 6.3828)
(8.52, 6.6562)
(8.87, 6.9297)
(9.19, 7.1797)
(9.52, 7.4375)
(9.82, 7.6719)
(10.11, 7.8984)
(10.39, 8.1172)
(10.68, 8.3437)
(15.35, 11.9922)

=== CASO 9 ===
alpha	f(mm)
(1.46, 1.1406)
(1.6, 1.2500)
(1.7, 1.3281)
(1.8, 1.4062)
(1.87, 1.4609)
(1.98, 1.5469)
(2.1, 1.6406)
(2.2, 1.7187)
(2.3, 1.7969)
(2.37, 1.8516)
(2.45, 1.9141)
(2.51, 1.9609)
(2.57, 2.0078)
(2.63, 2.0547)
(2.68, 2.0937)
(2.72, 2.1250)
(2.76, 2.